# Data Base Creation

Here firstly we are making a database file mydb.db from csv files. This database will furthur be used for querying various data and count them. This database file is made using our sample file so it will not reflect all of the data of New delhi.

In [2]:
import sqlite3
import csv
from pprint import pprint
import pandas as pd

sqlite_file = 'mydb.db'    # name of the sqlite database file

# Connect to the database
conn = sqlite3.connect(sqlite_file)

# Get a cursor object
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
conn.commit()

# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT,type TEXT)
''')
# commit the changes
conn.commit()

# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"),i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]

# insert the formatted data
cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()    

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes''')
conn.commit()

cur.execute('''
    CREATE TABLE nodes(id INTEGER, lat INTEGER, lon INTEGER,user TEXT, uid INGETER, version INTEGER, changeset TEXT,timestamp TEXT)
''')
# commit the changes
conn.commit()
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [ (i["id"], i['lat'],i['lon'],i['user'].decode('utf-8'), i['uid'], i["version"], i["changeset"], i["timestamp"]) for i in dr]
    
# insert the formatted data
cur.executemany("INSERT INTO nodes(id, lat, lon,user,uid,version,changeset,timestamp) VALUES (?, ?, ?, ? ,? ,? ,?,?);", to_db)
# commit the changes
conn.commit()

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways''')
conn.commit()

cur.execute('''
    CREATE TABLE ways(id INTEGER, user TEXT, uid INGETER, version INTEGER, changeset TEXT,timestamp TEXT)
''')
# commit the changes
conn.commit()

with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [ (i["id"], i['user'].decode('utf-8'), i['uid'], i["version"], i["changeset"], i["timestamp"]) for i in dr]
# insert the formatted data
cur.executemany("INSERT INTO ways(id,user,uid,version,changeset,timestamp) VALUES (?, ?, ?, ? ,? ,? );", to_db)
# commit the changes
conn.commit()
    
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways_tags''')
conn.commit()

cur.execute('''
CREATE TABLE ways_tags(id INTEGER, key TEXT, value TEXT, type TEXT) ''')
# commit the changes
conn.commit()
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [ (i["id"], i["key"],i["value"], i["type"]) for i in dr]
# insert the formatted data
cur.executemany("INSERT INTO ways_tags(id,key,value,type) VALUES (?, ?, ?, ? );", to_db)
# commit the changes
conn.commit()

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()

cur.execute('''
CREATE TABLE ways_nodes(id INTEGER, node_id TEXT, position INTEGER) ''')
# commit the changes
conn.commit()
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [ (i["id"], i["node_id"],i["position"]) for i in dr]
# insert the formatted data
cur.executemany("INSERT INTO ways_nodes(id,node_id,position) VALUES (?, ?, ?);", to_db)
# commit the changes
conn.commit()

# Queries

In [3]:
#number of "node" tags
import sqlite3
db = sqlite3.connect("mydb.db")
c = db.cursor()
QUERY = '''SELECT count(*)as num from nodes
'''
c.execute(QUERY)
rows = c.fetchall()
    
df = pd.DataFrame(rows)
print("number of nodes are " ,df)

db.close()

('number of nodes are ',         0
0  113875)


In [4]:
# number of 'Ways' tags
import sqlite3
db = sqlite3.connect("mydb.db")
c = db.cursor()
QUERY = '''SELECT count(*) as num from ways
'''
c.execute(QUERY)
rows = c.fetchall()
    
df = pd.DataFrame(rows)
print("number of ways are ",df)

db.close()

('number of ways are ',        0
0  23177)


In [5]:
#Top 10 amenities  available in New Delhi
import sqlite3
db = sqlite3.connect("mydb.db")
c = db.cursor()
QUERY = '''SELECT value,count(*)as num from (select value,key from nodes_tags  UNION ALL select value,key from ways_tags)
where key='amenity'
group by value
order by num desc
limit 10;
'''
c.execute(QUERY)
rows = c.fetchall()

d = pd.DataFrame(rows)
print("Top 10 amenities  available in New Delhi are ")
print(d)

db.close()

Top 10 amenities  available in New Delhi are 
                  0   1
0            school  21
1              fuel  13
2           parking   8
3               atm   7
4            cinema   6
5         fast_food   6
6          pharmacy   6
7  place_of_worship   5
8       marketplace   4
9          hospital   3


In [6]:
# number of unique users with list
import sqlite3
db = sqlite3.connect("mydb.db")
c = db.cursor()
QUERY = '''SELECT DISTINCT(user) from (select user from nodes UNION ALL select user from ways)'''

c.execute(QUERY)
rows = c.fetchall()
import pandas as pd    
df = pd.DataFrame(rows)
print("number of unique users are ",len(df))
print(df)

db.close()

('number of unique users are ', 625)
                           0
0                     rene78
1                    H_S_Rai
2                   PlaneMad
3                      sahil
4                   thevikas
5                   Oberaffe
6                  Neerajgis
7                     aarthy
8              marek kleciak
9                    roemcke
10            Nishant Sharma
11                    Edolis
12                  saikumar
13                  anthony1
14    OSMF Redaction Account
15                  pvprasad
16                    n'garh
17                   cdavila
18                   lxbarth
19                  praveeng
20                 CloCkWeRX
21                 hareesh11
22                  Apreethi
23               sourabh2910
24                    clgowa
25             SanjeevKumarT
26                   tsk1979
27                     Alban
28                    Mettal
29                ravikumar1
..                       ...
595               ChrissW-R1
596   

In [7]:
#cusines available and their count 
import sqlite3
db = sqlite3.connect("mydb.db")
c = db.cursor()
QUERY = '''SELECT value,count(*)as num from (select value,key from nodes_tags  UNION ALL select value,key from ways_tags)
where key='cuisine'
group by value
order by num desc
;
'''
c.execute(QUERY)
rows = c.fetchall()
import pandas as pd    
df = pd.DataFrame(rows)
print("Number of cuisines available in New Delhi are :", len(df))
print(df)

db.close()

('Number of cuisines available in New Delhi are :', 2)
              0  1
0        indian  6
1  multicuisine  2


In [9]:
#Top 10 contributers
import sqlite3
db = sqlite3.connect("mydb.db")
c = db.cursor()
QUERY = '''SELECT user,count(user)  from (select user from nodes UNION ALL select user from ways)
group by user
order by  count(user) desc
limit 10
'''
c.execute(QUERY) 
top = c.fetchall()
df=pd.DataFrame(top)
print("Top 10 contributers")
print (df)

Top 10 contributers
               0     1
0       Oberaffe  8978
1      premkumar  5453
2       saikumar  5390
3       Naresh08  4564
4        anushap  4457
5         sdivya  4320
6       anthony1  4195
7     himabindhu  4112
8  sathishshetty  4063
9       Apreethi  3796
